In [1]:
# Testing Pacman
import postprocess as pp
import plots as plots


postproc


In [2]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

## Stability of a discrete system

In [ ]:
# Run the simulation:

# !../practice/discrete-atk.py -N 2

In [ ]:
N

In [5]:
experiment = '../../practice/output/discrete-atk-N2'
params, data, signature = pp.load_data(experiment)
model_parameters = params.get("model")

_mu, _w1, _k, _N = model_parameters.get("mu"), model_parameters.get("w1"), model_parameters.get("k"), model_parameters["N"]


In [ ]:
data;

In [ ]:
len(data.cone_data)

## N=2

In [ ]:
import sympy as sp

In [32]:
## Simple Analytic 
α, α1, α2, e1, e2 = sp.symbols('α α1 α2 e1 e2')
u1, u2 = sp.symbols('u1 u2')
K1, K2 = sp.symbols('K1 K2', positive=True)
s1, s2 = sp.symbols('s1 s2', positive=True)
μ, N = sp.symbols('μ N', positive=True)
w1 = sp.symbols('w1', positive=True)
t = sp.Symbol('t', positive=True)
k = sp.Symbol('k', positive=True)

# _N = 2

K = lambda α: (1-α)/(1+(k-1)*α)
s = lambda α: 1/K(α)

U = 1/2*μ*(K1*u1**2 + K2*(t-u1)**2) * N

S = w1*(α1 + α2) / N

sol_u = sp.solve(U.diff(u1), u1)


matpar = {k: _k, w1: _w1, N: _N, μ: _mu}


In [33]:
U

0.5*N*μ*(K1*u1**2 + K2*(t - u1)**2)

In [34]:
sp.simplify(sol_u[0])

K2*t/(K1 + K2)

In [35]:
sp.simplify(sol_u[0]).subs({K1: 1/s(α), K2: 1/s(α)})

t/2

In [19]:
E=U.subs(u1, sol_u[0]).simplify()
E

0.5*K1*K2*N*t**2*μ/(K1 + K2)

In [20]:
E.subs({K1: 1/s1, K2: 1/s2}).simplify()

0.5*N*t**2*μ/(s1 + s2)

In [21]:
_E = E.subs({K1: 1/s(α), K2: 1/s(α)})
_E

0.25*N*t**2*μ*(1 - α)/(α*(k - 1) + 1)

In [22]:
F = (E+S)
# .subs({K1: 1/s(α), K2: 1/s(α)}).subs({})
F = (E+S).subs({K1: 1/s(α), K2: 1/s(α), α1: α, α2: α})

F

0.25*N*t**2*μ*(1 - α)/(α*(k - 1) + 1) + 2*w1*α/N

In [23]:
Fa = F.diff(α).simplify()
Fa

-0.25*N*k*t**2*μ/(k*α - α + 1)**2 + 2*w1/N

In [24]:
Fa.subs({α: 0})

-0.25*N*k*t**2*μ + 2*w1/N

In [26]:
sp.solve(Fa, t)[1].subs({α: 0})

2.82842712474619*sqrt(w1)/(N*sqrt(k)*sqrt(μ))

In [27]:
data.load.values

array([0.        , 0.06896552, 0.13793103, 0.20689655, 0.27586207,
       0.34482759, 0.4137931 , 0.48275862, 0.55172414, 0.62068966,
       0.68965517, 0.75862069, 0.82758621, 0.89655172, 0.96551724,
       1.03448276, 1.10344828, 1.17241379, 1.24137931, 1.31034483,
       1.37931034, 1.44827586, 1.51724138, 1.5862069 , 1.65517241,
       1.72413793, 1.79310345, 1.86206897, 1.93103448, 2.        ])

In [29]:
F.subs(matpar)

NameError: name 'matpar' is not defined

In [ ]:
Ef = sp.lambdify((α, t), F.subs(matpar), 'numpy')
Ef
# Ef(0, data.load.values)


In [ ]:
plt.plot(data.load, Ef(0, data.load.values))
plt.plot(data.load, data.elastic_energy)
plt.plot(data.load, _mu / 2. * data.load**2*1. / (s(0)+s(0)), ls='dotted', lw=4)
# plt.plot(data.load, _mu / 2. * data.load**2*1. / (s(0)+s(0)), ls='dotted', lw=4)

### Elasticity and Verification

In [ ]:
# model_parameters = params.get("model")
# _σc = np.sqrt(2*_mu*_w1/_k)
# _tc = np.sqrt(8*_w1/(_k*_mu)/_N**2)
# __tc = _tc.subs({_N: N})
# print(f'σ_c = {_σc}')
# print(f't_c = {_tc}')


In [ ]:
_tc = sp.solve(Ea, t)[1].subs({α: 0})
_tc
# __tc = _tc.subs(matpar)
# sp.N(__tc)

In [ ]:
sp.N(1/N*sp.sqrt(w1/k/μ).subs(matpar))

In [ ]:
print(_mu, _w1, _k)

In [ ]:
params, data, signature = pp.load_data(experiment)

plt.plot(data.load, data.fracture_energy, label = 'fractuc')
plt.plot(data.load, data.elastic_energy, label = 'elastic')
_total = data.fracture_energy + data.elastic_energy
plt.plot(data.load, _total, label = 'total')
plt.axvline(__tc, c='k')
# plt.axvline(_tc/2, c='k', ls='dotted')
plt.legend()
plt.xlabel('load')
plt.ylabel('Energy')

plt.plot(data.load, Ef(0, data.load.values), ls = 'dotted')
# plt.plot(data.load, 2*Ef(0, data.load.values), ls = 'dotted')



In [ ]:
plt.plot(data.load, data.F, label = 'stress')
# plt.plot(data.load, data.elastic_energy, label = 'elastic')
# _total = data.fracture_energy + data.elastic_energy
# plt.plot(data.load, _total, label = 'total')
plt.legend()
plt.xlabel('load')
plt.ylabel('stress')
# plt.axhline(_σc/2, c='k')

In [ ]:
for k in range(model_parameters["N"]):
    plt.plot(data.load, [a[k] for a in data.alpha_t], label=f'$\\alpha_{k}$')

plt.title("$\\alpha$")
plt.legend()
plt.axhline(1, c='k')

In [ ]:
for k in range(model_parameters["N"]+1):
    plt.plot(data.load, [u[k] for u in data.u_t], label=f'$u_{k}$')
plt.plot(data.load, 1/_mu*data.load/2, label=f'$u^*$', ls='dotted', lw=4)

plt.legend()
plt.title("$u$")
plt.axhline(data.load.values[-1], c='k')

In [ ]:
params;

In [ ]:
for i in range(len(data.cone_data)):
    plt.plot(data.cone_data[i]['error_x_L2'], label=f'step {i}')
plt.semilogy()
# plt.legend()
plt.xlabel('iterations')
plt.ylabel('$||\Delta x_k||_{L^2}$')


In [ ]:
data.cone_data[3].keys()

In [ ]:
for i in range(len(data.cone_data)):
    plt.plot(data.cone_data[i]['y_norm_L2'], label=f'step {i}')
# plt.semilogy()
# plt.legend()
plt.axhline(0, c='k')
plt.xlabel('iterations')
plt.ylabel('$||y_k||_{L^2}$')


In [ ]:
for i in range(len(data.cone_data)):
    plt.plot(data.cone_data[i]['lambda_k'], label=f'step {i}')
plt.axhline(0, c='k')
plt.title('Convergence of eigenvalues')
plt.xlabel('iterations')


In [ ]:
data.cone_data

In [ ]:
data.cone_data[3]['iterations']

In [ ]:
[converged['iterations'] for converged in data.cone_data]
# [converged['lambda_k'] for converged in data.cone_data]

In [ ]:
asd = np.array([converged['lambda_k'][-1] for converged in data.cone_data if isinstance(converged['iterations'], int)])
asd
# plt.plot(asd)

In [ ]:
[converged['lambda_k'][-1] for converged in data.cone_data if len(converged['lambda_k'])>0]

## Stability of a short bar

In [ ]:
!ls ../../playground/

In [ ]:
experiment = '../../practice/output/traction_cone'
params, data, signature = pp.load_data(experiment)


In [ ]:
data.cone_data[6]

In [ ]:
[data.cone_data[i]['iterations'] for i in range(len(data.cone_data))]

$$
s:= t + \int_0^t ||\dot \alpha||_{1, 2} dx
$$

In [ ]:
data.get("rate_12_norm")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
params.get("geometry").get("Ly")

In [ ]:
sell = []
_dt = (params.get("loading").get("max") - params.get("loading").get("min")) /params.get("loading").get("steps")
for i, t in enumerate(data.get("load")):
    sell.append(np.trapz(data.get("rate_12_norm")[0:i], data.get("load")[0:i])/_dt)

s0 = []
_dt = (params.get("loading").get("max") - params.get("loading").get("min")) /params.get("loading").get("steps")
for i, t in enumerate(data.get("load")):
    s0.append(np.trapz(data.get("rate_12_norm_unscaled")[0:i], data.get("load")[0:i])/_dt)
    
# s = data.get("load") + s0

In [ ]:
plt.title("Evolution Jumps?")
# plt.plot(data.get("load"), s, marker = 'o', label='s')
plt.plot(data.get("load"), sell, marker = 'o', label='$s_\ell$')
plt.plot(data.get("load"), s0, marker = 'o', label='$s_0$')
plt.plot(data.get("load"), data.get("load"), marker = 'o', label='t')
plt.ylabel('$t, s_0, s$')
plt.xlabel('t')
plt.legend()
plt.twinx()
plt.plot(data.get("load"), data.get("fracture_energy")/params.get("geometry").get("Ly"),
    marker = 's', c='k', label='fracture')
plt.ylabel('energy')
plt.legend(loc=4)
plt.savefig('jumps_evo_multifiss.png')

In [ ]:
plt.plot(data.get("load"), sell, marker = 'o', label='$s_\ell$')
plt.twinx()
plt.plot(np.diff(np.array(sell)), label="")
plt.xlabel('t')
